## Compulsory Task 

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [107]:
# Load up store_income_data.csv

import pandas as pd
import numpy as np

df = pd.read_csv('store_income_data_task.csv', index_col = 0) 


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [108]:
# Looking at the number of unique values in the country column.
unique_countries_count = df["country"].nunique()
print(f"There are {unique_countries_count} unique countries")

# Converting the column into lowercase.
df["country"] = df["country"].str.lower()

# Removing any whitespace.
df["country"] = df["country"].str.strip()



There are 76 unique countries


2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [109]:
# Removing specified characters and converting to lowercase
df["country"] = df["country"].str.replace(r'[./\\]', '').str.lower()

# Mapping variations to standardized country names
country_mapping = {
    'south africa': 'South Africa',
    'SA': 'South Africa',
    'S Africa' : 'South Africa',
    'S. AfricaSouth Africa' : 'South Africa',
    'S. AfricaSouth Africa' : 'South Africa',
    'united kingdom': 'United Kingdom',
    'britian' : 'United Kingdom',
    'uk' : 'United Kingdom',
    'u k' : 'United Kingdom',
    'england' : 'United Kingdom',
    'UNITED STATES OF AMERICA' : 'United States',
    'united states': 'United States',
    'america' : 'United States',
    "united states of america" : 'United States',
    'usa' : 'United States'
}

# Applying the mapping to the "country" column
df["country"] = df["country"].str.lower().map(country_mapping)


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [111]:
import pandas as pd
import datetime

# Create a copy of the DataFrame to avoid modifying the original data
df_copy = df.copy()

# Convert the 'date_measured' column to datetime format
df_copy['date_measured'] = pd.to_datetime(df['date_measured'], errors='coerce')

# Calculate the difference in days for each date from the current date
current_date = datetime.date.today()
df_copy['days_ago'] = (current_date - df_copy['date_measured'].dt.date)

# Print the DataFrame to see the new column
print(df_copy).head()



                                             store_name           store_email  \
id                                                                              
1                            Cullen/Frost Bankers, Inc.                   NaN   
2                                   Nordson Corporation                   NaN   
3                                 Stag Industrial, Inc.                   NaN   
4                                   FIRST REPUBLIC BANK    ecanadine3@fc2.com   
5                           Mercantile Bank Corporation                   NaN   
...                                                 ...                   ...   
996                         Columbia Sportswear Company  cschooleyrn@sohu.com   
997   WisdomTree Interest Rate Hedged High Yield Bon...                   NaN   
998          Tortoise Energy Infrastructure Corporation   cbeardshallrp@ow.ly   
999                                   Qwest Corporation                   NaN   
1000                        

AttributeError: 'NoneType' object has no attribute 'head'